In [53]:
#%matplotlib inline

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import time as t
from sklearn import preprocessing as pre

In [3]:
import import_datasets as ids
datasets=ids.importData()

In [4]:
import utility as u
join=u.joinTable()
join.head()

,fault_severity,id,location,sample,severity_type,resource_type,log_feature,volume,event_type
0,1,14121,location 118,train,severity_type 2,resource_type 2,feature 312,4,event_type 34
1,1,14121,location 118,train,severity_type 2,resource_type 2,feature 312,4,event_type 35
2,1,14121,location 118,train,severity_type 2,resource_type 2,feature 232,4,event_type 34
3,1,14121,location 118,train,severity_type 2,resource_type 2,feature 232,4,event_type 35
4,0,9320,location 91,train,severity_type 2,resource_type 2,feature 315,4,event_type 34


In [9]:
ndf,dr=u.drawCT(join,'log_feature')
plt.show()

In [7]:
a

log_feature
feature 104    0.000000
feature 107    0.000000
feature 109    0.000000
feature 11     0.000000
feature 110    0.000000
feature 122    0.000000
feature 123    0.000000
feature 128    0.000000
feature 130    0.000000
feature 131    0.000000
feature 142    0.000000
feature 143    0.000000
feature 16     0.000000
feature 175    0.000000
feature 18     0.000000
feature 180    0.000000
feature 192    0.000000
feature 194    0.000000
feature 21     0.000000
feature 213    0.000000
feature 215    0.000000
feature 216    0.000000
feature 226    0.000000
feature 231    0.000000
feature 236    0.000000
feature 24     0.000000
feature 242    0.000000
feature 243    0.000000
feature 244    0.000000
feature 245    0.000000
                 ...   
feature 91     0.360000
feature 92     0.360000
feature 108    0.000000
feature 158    0.333333
feature 43     0.333333
feature 69     0.217391
feature 288    0.000000
feature 238    0.000000
feature 200    0.166667
feature 13     0.000000
feat

In [6]:
from scipy.stats import chisquare
dict_stats={}
cat_cols=['location','log_feature','severity_type','resource_type','event_type']

for column in cat_cols:
    df=pd.crosstab(join[column],join['fault_severity'])
    df[column+'_sum']=df.sum(axis=1)
    df.columns=['0','1','2',column+'_sum']
    df[column+'_std']=df[['0','1','2']].std(axis=1)
    f,_=chisquare(df[['0','1','2']],axis=1)
    df[column+'_pchisqr']=f
    
    fs=[column+'_sum',column+'_std',column+'_pchisqr']
    df=df[fs]
    df[column]=df.index
    dict_stats[column]=df
    join=pd.merge(join,df,on=column,how="left") #safer than inner

In [26]:
stats=join.iloc[:,9:]
stats['id']=join['id']
stats=stats.groupby('id').mean()
stats=stats.fillna(value=0)
stats['id']=stats.index
for column in stats.columns:
    if column is not 'id':
        stats[column]=pd.qcut(x=stats[column],q=7,labels=[1,2,3,4,5,6,7])

ValueError: Bin edges must be unique: array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.])

In [27]:
stats

,location_location 1004,location_location 1005,location_location 1006,location_location 1007,location_location 1008,location_location 1009,location_location 101,location_location 1010,location_location 1011,location_location 1012,...,severity_type_sum_y,severity_type_std_y,severity_type_pchisqr_y,resource_type_sum_y,resource_type_std_y,resource_type_pchisqr_y,event_type_sum_y,event_type_std_y,event_type_pchisqr_y,id
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0.861703,-0.770879,-0.919725,-1.816062,-1.098482,-0.978311,-0.450878,-0.720537,-0.745905,1
2,0,0,0,0,0,0,0,0,0,0,...,-0.482820,1.017888,1.058727,1.017233,1.127212,1.129855,0.603482,1.348284,1.389013,2
3,0,0,0,0,0,0,0,0,0,0,...,0.861703,-0.770879,-0.919725,-0.606435,-0.893324,-0.924536,0.990179,-0.339927,-0.639137,3
4,0,0,0,0,0,0,0,0,0,0,...,-3.031855,-1.985095,-1.120562,1.017233,1.127212,1.129855,-2.510436,-1.213997,-0.874740,4
5,0,0,0,0,0,0,0,0,0,0,...,-0.482820,1.017888,1.058727,1.017233,1.127212,1.129855,0.603482,1.348284,1.389013,5
6,0,0,0,0,0,0,0,0,0,0,...,-0.482820,1.017888,1.058727,1.017233,1.127212,1.129855,0.522802,1.277550,1.322384,6
7,0,0,0,0,0,0,0,0,0,0,...,0.861703,-0.770879,-0.919725,1.017233,1.127212,1.129855,-2.286311,-1.165772,-0.863575,7
8,0,0,0,0,0,0,0,0,0,0,...,-0.482820,1.017888,1.058727,1.017233,1.127212,1.129855,0.603482,1.348284,1.389013,8
9,0,0,0,0,0,0,0,0,0,0,...,0.861703,-0.770879,-0.919725,-0.606435,-0.893324,-0.924536,-0.040206,-0.714504,-0.751576,9


In [28]:
'resource_type_pchisqr_x' in join.columns

True

In [18]:
datasets['stats']=stats
join=u.newJoin(datasets)
join.head()

,fault_severity,id,location_location 1,location_location 10,location_location 100,location_location 1000,location_location 1001,location_location 1002,location_location 1003,location_location 1004,...,location_location 995,location_location 996,location_location 997,location_location 998,location_location 999,location_nan,sample,sample_nan,sample_test,sample_train
0,1,1,0,0,0,0,NaN,0,NaN,NaN,...,0,0,NaN,0,0,0,train,0,NaN,1
1,0,5,0,0,0,0,NaN,0,NaN,NaN,...,0,0,NaN,0,0,0,train,0,NaN,1
2,1,6,0,0,0,0,NaN,0,NaN,NaN,...,0,0,NaN,0,0,0,train,0,NaN,1
3,0,8,0,0,0,0,NaN,0,NaN,NaN,...,0,0,NaN,0,0,0,train,0,NaN,1
4,0,13,0,0,0,0,NaN,0,NaN,NaN,...,0,0,NaN,0,0,0,train,0,NaN,1


In [20]:
#convert table of ids and classes to binary row feature vectors for each unique id
for key, dataset in datasets.items():
    if key is not 'stats':
        tmp=pd.get_dummies(dataset,dummy_na=True)
        tmp=tmp.groupby('id').sum()
        tmp['id']=tmp.index
        dataset=tmp
        datasets[key]=dataset

In [21]:
#joins features from different datasets into one dataframe, features only for evaluated ids are extracted
#(i.e. some features may be not related to any id)
samples=['train','test']
for key, dataset in datasets.items():
    if key not in samples:
        join=pd.merge(join,dataset,on='id',how='left')

join=pd.merge(join,stats,on='id')
join=join.set_index(['sample'])
join=pd.get_dummies(join) #fixes dummies for location

In [23]:
join.head()

,fault_severity,id,location_location 1,location_location 10,location_location 100,location_location 1000,location_location 1001,location_location 1002,location_location 1003,location_location 1004,...,log_feature_pchisqr_y,severity_type_sum_y,severity_type_std_y,severity_type_pchisqr_y,resource_type_sum_y,resource_type_std_y,resource_type_pchisqr_y,event_type_sum_y,event_type_std_y,event_type_pchisqr_y
sample,,,,,,,,,,,,,,,,,,,,,
train,1,1,0,0,0,0,NaN,0,NaN,NaN,...,-0.682107,0.861703,-0.770879,-0.919725,-1.816062,-1.098482,-0.978311,-0.450878,-0.720537,-0.745905
train,0,5,0,0,0,0,NaN,0,NaN,NaN,...,2.209474,-0.482820,1.017888,1.058727,1.017233,1.127212,1.129855,0.603482,1.348284,1.389013
train,1,6,0,0,0,0,NaN,0,NaN,NaN,...,0.887044,-0.482820,1.017888,1.058727,1.017233,1.127212,1.129855,0.522802,1.277550,1.322384
train,0,8,0,0,0,0,NaN,0,NaN,NaN,...,0.703080,-0.482820,1.017888,1.058727,1.017233,1.127212,1.129855,0.603482,1.348284,1.389013
train,0,13,0,0,0,0,NaN,0,NaN,NaN,...,2.209474,-0.482820,1.017888,1.058727,1.017233,1.127212,1.129855,0.603482,1.348284,1.389013


In [24]:
join.columns

Index(['fault_severity', 'id', 'location_location 1', 'location_location 10',
       'location_location 100', 'location_location 1000',
       'location_location 1001', 'location_location 1002',
       'location_location 1003', 'location_location 1004', 
       ...
       'log_feature_pchisqr_y', 'severity_type_sum_y', 'severity_type_std_y',
       'severity_type_pchisqr_y', 'resource_type_sum_y', 'resource_type_std_y',
       'resource_type_pchisqr_y', 'event_type_sum_y', 'event_type_std_y',
       'event_type_pchisqr_y'],
      dtype='object', length=1621)

In [37]:
#normalizes a continuous feature
#join['volume']=pre.minmax_scale(join['volume'].astype(float))

#fills in missing data
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
join = DataFrameImputer().fit_transform(join)
#join=join.fillna(value=0)

In [50]:
from kmodes import kmodes

km = kmodes.KModes(n_clusters=5, init='Huang', n_init=1, verbose=1)
clusters = km.fit_predict(join[features])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 4520, cost: 167141.0
Run 1, iteration: 2/100, moves: 3732, cost: 163281.0
Run 1, iteration: 3/100, moves: 905, cost: 163281.0


In [72]:
join['clusters']=clusters
join=pd.concat([join,pd.get_dummies(join['clusters'],prefix='cluster')],axis=1)
del join['clusters']
join.head()

,fault_severity,id,p0,p1,p2,sum,volume,log_feature_feature 1,log_feature_feature 10,log_feature_feature 100,...,location_location 995,location_location 996,location_location 997,location_location 998,location_location 999,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4
sample,,,,,,,,,,,,,,,,,,,,,
train,1,14121,0.909091,0.090909,0.000000,33,0.094788,0,0,0,...,0,0,0,0,0,0,0,1,0,0
train,0,9320,0.897959,0.102041,0.000000,49,3.670196,0,0,0,...,0,0,0,0,0,0,0,1,0,0
train,1,14394,0.600000,0.400000,0.000000,5,-0.368214,0,0,0,...,0,0,0,0,0,0,0,1,0,0
train,1,8218,0.696970,0.181818,0.121212,33,-0.110991,0,0,0,...,0,0,0,0,0,0,0,0,0,1
train,0,14804,0.500000,0.500000,0.000000,12,-0.239603,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [38]:
#creates features and targets for final training
features=join.columns.values.tolist()
not_features=['fault_severity', 'id'] #not_features=['fault_severity', 'id'] 
for x in features:
    if "_nan" in x:
        features.remove(x)
        not_features.append(x)
        
for x in not_features:
    if x in features:
        features.remove(x)

In [39]:
train_features = join.loc['train'][features]
train_target = join.loc['train']['fault_severity']

test_features = join.loc['test'][features]

In [40]:
import xgboost as xgb

sz=(train_features.values).shape

train_X = train_features.values[:int(sz[0] * 0.7), :]
train_Y = train_target.values[:int(sz[0]*0.7)]
#wtr = join.loc['train']['sum'].values[:int(sz[0]*0.7)]


test_X = train_features.values[int(sz[0] * 0.7):, :]
test_Y = train_target.values[int(sz[0] * 0.7):]
#wte = join.loc['train']['sum'].values[int(sz[0] * 0.7):]

xg_train = xgb.DMatrix(train_X, label=train_Y,feature_names=train_features.columns) #
#xg_train.set_weight(wtr)
full_xg_train = xgb.DMatrix(train_features, label=train_target.values,feature_names=train_features.columns) #

xg_test = xgb.DMatrix(test_X, label=test_Y,feature_names=test_features.columns) #
#xg_test.set_weight(wte)
full_xg_test = xgb.DMatrix(test_features,feature_names=test_features.columns) #

In [45]:
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob'
param['eta'] = 1
param['gamma']= 4
param['max_depth'] = 15
param['min_child_weight']=4
param['max_delta_step']=4
param['subsample']=0.5

param['nthread'] = 3
#param['subsample']=1
param['num_class'] = 3
param['eval_metric']='mlogloss'

watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 4
bst = xgb.train(param, xg_train,num_round,evals=watchlist);
# get prediction
pred = bst.predict( xg_test );

xgb.cv(param, xg_train, num_round, nfold=3,
       metrics=['mlogloss'], seed = 0)

[0]	train-mlogloss:0.621968	test-mlogloss:0.672761
[1]	train-mlogloss:0.531834	test-mlogloss:0.607101
[2]	train-mlogloss:0.504729	test-mlogloss:0.589246
[3]	train-mlogloss:0.492034	test-mlogloss:0.578604


,test-mlogloss-mean,test-mlogloss-std,train-mlogloss-mean,train-mlogloss-std
0,0.675589,0.009442,0.621989,0.005857
1,0.610774,0.012241,0.535966,0.006149
2,0.590008,0.016091,0.503040,0.003222
3,0.582110,0.016329,0.482687,0.003750


In [ ]:
xgb.plot_importance(booster=bst)
plt.show()

In [27]:
bst=xgb.train(param, full_xg_train, num_round,evals=watchlist)
test_response=bst.predict(full_xg_test)
xgb.cv(param, xg_train, num_round, nfold=3,
       metrics=['mlogloss'], seed = 0)

[0]	train-mlogloss:0.618051	test-mlogloss:0.613721
[1]	train-mlogloss:0.543451	test-mlogloss:0.536194
[2]	train-mlogloss:0.515183	test-mlogloss:0.510487
[3]	train-mlogloss:0.492402	test-mlogloss:0.489787


,test-mlogloss-mean,test-mlogloss-std,train-mlogloss-mean,train-mlogloss-std
0,0.668221,0.006264,0.633977,0.010597
1,0.603008,0.012130,0.559228,0.007504
2,0.581622,0.013304,0.532272,0.006528
3,0.570103,0.013579,0.511354,0.008957


In [24]:
test_response

array([ 0.,  2.,  0., ...,  1.,  0.,  0.], dtype=float32)

In [28]:
#save predictions to required format 
#make it binary?
out=pd.DataFrame({'id':[],'predict_0':[],'predict_1':[],'predict_2':[]})
out['id']=join.loc['test']['id']
out['predict_0']=test_response.T[0]
out['predict_1']=test_response.T[1]
out['predict_2']=test_response.T[2]
out=out[['id','predict_0','predict_1','predict_2']]
out.to_csv('predictions.csv',index=False)

In [60]:
for column in join.columns:
    if column not in ['id','fault_severity']:
        join[column]=join[column].astype(np.float32)

In [120]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import RidgeClassifierCV, Perceptron, SGDClassifier, PassiveAggressiveClassifier
from sklearn.dummy import DummyClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.svm import LinearSVC
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier, OutputCodeClassifier
from sklearn import grid_search

In [77]:
train_x, test_x, train_y, test_y = train_test_split(train_features, train_target, test_size=0.31, random_state=0)
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = t.time()
    #clf=OneVsRestClassifier(clf) #may want to remove for testing
    clf.fit(train_x, train_y)
    train_time = t.time() - t0
    print("Training time: %0.3fs" % train_time)

    t0 = t.time()
    y_pred_single = clf.predict(test_x)
    if (hasattr(clf,'predict_proba')):
        y_pred_dist = clf.predict_proba(test_x)
        logloss=log_loss(test_y,y_pred_dist)
        print("LogLoss: %0.3fs" % logloss)
    test_time = t.time() - t0
    print("Prediction time:  %0.3fs" % test_time)

    acc_score = accuracy_score(test_y, y_pred_single)
    print("Accuracy:   %0.3f" % acc_score)
    
    #if (hasattr(clf,'predict_proba')):
       #scores = cross_val_score(clf,train_features,train_target, scoring='log_loss')
        #loss_score=scores.mean()
        #print("CV LogLoss: %0.2f (+/- %0.2f)" % (loss_score, scores.std() * 2))
    
    print(classification_report(test_y, y_pred_single))
    clf_descr = str(clf).split('(')[0]
    
    if (hasattr(clf,'predict_proba')):
        return clf_descr, acc_score, logloss, train_time, test_time
    else:
        return clf_descr, acc_score, train_time, test_time

In [79]:
results = []
for clf, name in (
        (RidgeClassifierCV(), "Ridge Classifier"),
        (RandomForestClassifier(n_estimators=300), "Random forest"),
        (AdaBoostClassifier(), "AdaBoost"),
        (BaggingClassifier(), "Bagging"),
        (QuadraticDiscriminantAnalysis(), "DiscriminatnAnalysis")):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf))

Ridge Classifier
________________________________________________________________________________
Training: 
RidgeClassifierCV(alphas=(0.1, 1.0, 10.0), class_weight=None, cv=None,
         fit_intercept=True, normalize=False, scoring=None)
Training time: 4.643s
Prediction time:  0.038s
Accuracy:   0.727
             precision    recall  f1-score   support

        0.0       0.79      0.91      0.84      1476
        1.0       0.59      0.35      0.44       593
        2.0       0.49      0.55      0.52       220

avg / total       0.71      0.73      0.71      2289

Random forest
________________________________________________________________________________
Training: 
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verb

C:\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:688: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [115]:
#test a classifier

# Fit the model to training data
#parameters={'n_estimators':[100,150]},'max_depth':[3]}
clf = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=200,verbose=1,presort=True,max_depth=7),n_jobs=-2)#,min_samples_leaf=10,min_samples_split=5)
#clf=grid_search.GridSearchCV(gbc, parameters,n_jobs=-2)
benchmark(clf)

________________________________________________________________________________
Training: 
OneVsRestClassifier(estimator=GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=7, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200, presort=True,
              random_state=None, subsample=1.0, verbose=1,
              warm_start=False),
          n_jobs=-2)
Training time: 92.315s
Prediction time:  0.573s
Accuracy:   0.739
             precision    recall  f1-score   support

        0.0       0.80      0.89      0.84      1476
        1.0       0.61      0.42      0.50       593
        2.0       0.55      0.60      0.58       220

avg / total       0.73      0.74      0.73      2289



('OneVsRestClassifier',
 0.73918741808650068,
 0.57045593736126277,
 92.31480956077576,
 0.5728719234466553)

In [122]:
#test a classifier

# Fit the model to training data
#parameters={'n_estimators':[100,150]},'max_depth':[3]}
clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=400,verbose=1,min_samples_split=10,\
                                                 oob_score=True,n_jobs=-2),n_jobs=-2)
#clf=grid_search.GridSearchCV(gbc, parameters,n_jobs=-2)
benchmark(clf)

________________________________________________________________________________
Training: 
OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-2,
            oob_score=True, random_state=None, verbose=1, warm_start=False),
          n_jobs=-2)
Training time: 31.099s

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks    


Prediction time:  1.980s
Accuracy:   0.751
             precision    recall  f1-score   support

        0.0       0.80      0.90      0.85      1476
        1.0       0.63      0.44      0.52       593
        2.0       0.59      0.56      0.57       220

avg / total       0.74      0.75      0.74      2289



('OneVsRestClassifier',
 0.75141983398864132,
 0.54245802948043198,
 31.099143743515015,
 1.9801154136657715)

In [110]:
scores = cross_val_score(clf,train_features,train_target, scoring='log_loss',n_jobs=-2,verbose=True)
print("LogLoss: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

LogLoss: -0.56 (+/- 0.01)


[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   44.6s finished


In [111]:
bestCLF=clf

In [112]:
bestCLF.fit(train_features,train_target)

[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-2)]: Done 194 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-2)]: Done 400 out of 400 | elapsed:   10.2s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=800,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-2,
            oob_score=True, random_state=None, verbose=1, warm_start=False)

In [113]:
test_response=bestCLF.predict_proba(test_features)

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.7s finished


In [5]:
#transform cat labels into digits
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for column in join.columns:
    if column not in notCat:
        join[column]=le.fit_transform(join[column])
    
join.head()

fault_severity 
 id 
 location 
 event_type 
 severity_type 
 resource_type 
 log_feature 
 volume 
 
 
 sample 
 
 
 
 
 
 
 
 
 
 
 
 
 
 train 
 0 
 1 
 14121 
 148 
 26 
 1 
 2 
 237 
 19 
 
 
 1 
 1 
 14121 
 148 
 26 
 1 
 2 
 148 
 19 
 
 
 2 
 1 
 14121 
 148 
 27 
 1 
 2 
 237 
 19 
 
 
 3 
 1 
 14121 
 148 
 27 
 1 
 2 
 148 
 19 
 
 
 4 
 0 
 9320 
 1027 
 26 
 1 
 2 
 240 
 200